In [1]:
import evaluate
import utils
import os

[2023-09-04 02:21:15,121] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
roc_auc_score = evaluate.load("roc_auc", "multiclass")

In [3]:
print(roc_auc_score.name)

rocauc


In [2]:
os.environ["TRANSFORMERS_OFFLINE"] = "1"

In [8]:
preds = [0, 1, -1]

# refs = [1, 1, 1]
refs = [0, 0, 0]

In [9]:
def test_metric(metric="f1"):
    if isinstance(metric, str):
        metric = evaluate.load(metric)
    print(metric.compute(predictions=preds, references=refs))
    print(metric.compute(predictions=preds, references=refs, average="micro"))

In [10]:
for metric_name in utils.CLF_METRIC_NAMES:
    try:
        test_metric(metric_name)
    except Exception as e:
        print(e)

Using the latest cached version of the module from /data/users/zhangjunlei/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Mon Aug 21 01:05:47 2023) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


{'accuracy': 0.3333333333333333}
Accuracy._compute() got an unexpected keyword argument 'average'


KeyboardInterrupt: 

In [7]:
test_metric("recall")

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [3]:
clf_metrics = evaluate.combine(["recall", "precision"])

Using the latest cached version of the module from /data/users/zhangjunlei/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--precision/4e7f439a346715f68500ce6f2be82bf3272abd3f20bdafd203a2c4f85b61dd5f (last modified on Thu Aug 31 03:24:17 2023) since it couldn't be found locally at evaluate-metric--precision, or remotely on the Hugging Face Hub.


In [8]:
from sklearn.metrics import accuracy_score

y_true = [0, 1, 2, 2, 1, 0]
y_pred = [0, 1, 2, 2, 0, 1]

# 计算多类别分类的准确率
accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.67


In [18]:
clf_metrics.compute(predictions=preds, references=refs, average=None, labels=[-1, 1, 0])

/data/users/zhangjunlei/anaconda3/envs/open-instruct/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'recall': array([0.        , 0.        , 0.33333333]),
 'precision': array([0., 0., 1.])}

In [14]:
clf_metrics.add_batch(predictions=preds, references=refs)

In [17]:
clf_metrics.add_batch(predictions=preds, references=refs)
for evaluation_module in clf_metrics.evaluation_modules:
    # print(evaluation_module._feature_names())
    print(evaluation_module.name)
    # batch = {"predictions": predictions, "references": references, **kwargs}
    # batch = {input_name: batch[input_name] for input_name in evaluation_module._feature_names()}
    # evaluation_module.add_batch(**batch)
    # metrics.append(evaluation_module)
    print(evaluation_module.compute(average=None, labels=[-1, 1, 0]))

recall
{'recall': array([0.        , 0.        , 0.33333333])}
precision
{'precision': array([0., 0., 1.])}


/data/users/zhangjunlei/anaconda3/envs/open-instruct/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
labels = [0, 1, -1]
eval_metrics = {}
all_predictions = [0, 1, -1]
all_references = [1, 1, 1]
metrics = clf_metrics
if isinstance(metrics, evaluate.CombinedEvaluations):
    metrics = metrics.evaluation_modules
for metric in metrics:
    if metric.name not in utils.MUL_CLF_METRIC_NAMES:
        eval_metrics.update(
            metric.compute(predictions=all_predictions, references=all_references)
        )
    else:  # metric.name in utils.MUL_CLF_METRIC_NAMES:
        for average in utils.MULTICLASS_AVERAGINGS:
            eval_metrics.update(
                {
                    f"{metric.name}_{average}": metric.compute(
                        predictions=all_predictions,
                        references=all_references,
                        average=average,
                    )[metric.name]
                }
            )
        class_metrics = metric.compute(
            predictions=all_predictions,
            references=all_references,
            average=None,
            labels=labels,
        )
        eval_metrics.update(
            {
                f"{metric.name}_{labels[idx]}": value
                for idx, value in enumerate(class_metrics[metric.name])
            }
        )

print(eval_metrics)

{'recall_micro': 0.3333333333333333, 'recall_macro': 0.1111111111111111, 'recall_weighted': 0.3333333333333333, 'recall_0': 0.0, 'recall_1': 0.3333333333333333, 'recall_-1': 0.0, 'precision_micro': 0.3333333333333333, 'precision_macro': 0.3333333333333333, 'precision_weighted': 1.0, 'precision_0': 0.0, 'precision_1': 1.0, 'precision_-1': 0.0}


/data/users/zhangjunlei/anaconda3/envs/open-instruct/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/users/zhangjunlei/anaconda3/envs/open-instruct/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/users/zhangjunlei/anaconda3/envs/open-instruct/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
